In [1]:
from train_data_processor import TrainDataProcessor
# from test_data_processor import TestDataProcessor
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
train_file_path = "data/train.csv"
train_data_processor = TrainDataProcessor(train_file_path)
train_df,columns_to_dropped = train_data_processor.run_process()

In [6]:
train_df.drop(columns=columns_to_dropped, inplace=True)
columns_to_drop = ["Universite Adi","Universite Kategori",'Lise Adi','Lise Sehir']
train_df.drop(columns=columns_to_drop, inplace=True)

In [8]:
from sklearn.impute import KNNImputer

# KNN Imputer kullanarak NaN değerlerini doldurma
imputer = KNNImputer(n_neighbors=5)
train_df = pd.DataFrame(imputer.fit_transform(train_df), columns=train_df.columns)

In [9]:
train_df.loc[train_df['Basvuru Yili'] == 2016, 'Degerlendirme Puani'] = train_df.loc[train_df['Basvuru Yili'] == 2016, 'Degerlendirme Puani'] / 2
train_df['Basvuru Yili'] = train_df['Basvuru Yili'] - 2000

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
# Veri setini özellikler (X) ve hedef değişken (y) olarak ayırma
X = train_df.drop('Degerlendirme Puani', axis=1)
y = train_df['Degerlendirme Puani']
# Eğitim ve test veri setlerine ayırma
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
from catboost import CatBoostRegressor
# CatBoostRegressor modelini oluştur
model = CatBoostRegressor(
    iterations=500,      # Modeli kaç iterasyon (ağaç) eğitmek istediğini belirtir
    learning_rate=0.1,    # Öğrenme oranı
    depth=6,              # Ağacın derinliği
    verbose=100           # Eğitim sürecinde her 100 iterasyonda bir çıktı verir
)


# Modeli eğit
model.fit(X_train, y_train)

# Tahmin yap
y_pred = model.predict(X_test)
# Model performansını değerlendirme (Root Mean Squared Error kullanarak)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
print(f"Root Mean Squared Error (RMSE): {rmse}")

0:	learn: 13.6400321	total: 167ms	remaining: 1m 23s
100:	learn: 4.6873469	total: 813ms	remaining: 3.21s
200:	learn: 4.3925639	total: 1.42s	remaining: 2.12s
300:	learn: 4.2363538	total: 2.02s	remaining: 1.33s
400:	learn: 4.1343799	total: 2.63s	remaining: 650ms
499:	learn: 4.0513838	total: 3.27s	remaining: 0us
Root Mean Squared Error (RMSE): 4.495676502911562
